In [129]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import BaggingClassifier, RandomForestRegressor, RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.ensemble StackingClassifier
from sklearn.metrics import f1_score, mean_absolute_error, roc_auc_score, make_scorer
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from catboost import CatBoostClassifier,Pool
from catboost.utils import get_confusion_matrix

ImportError: cannot import name 'StackingClassifier' from 'sklearn.ensemble' (C:\Users\GANSOR-PC\Anaconda3\lib\site-packages\sklearn\ensemble\__init__.py)

## 2. Ансамбли моделей. Бутстреппинг. Бэггинг

#### Задание 2.2
Вычислите дисперсию для этого ряда. Результат округлите до трёх знаков после точки-разделителя.

In [4]:
row = np.array([4.4, 3.8, 4.8, 4.0, 3.4, 4.2, 5.2])
var = (row**2).mean() - row.mean()**2
round(var,3), round(row.std()**2,3)

(0.317, 0.317)

### Задание 2.7
Объёмная и содержательная практика у нас ещё впереди, но в качестве разминки давайте поработаем с уже известным вам датасетом о вине, который можно скачать [здесь](https://lms.skillfactory.ru/assets/courseware/v1/805b5c231251e174abb4fdbbd391adc3/asset-v1:SkillFactory+DSPR-2.0+14JULY2021+type@asset+block/wineQualityReds.zip).

Ранее вы обучали на данных только один алгоритм, а теперь мы попробуем сравнить несколько.

Подготовьте данные к классификации. Условно разделите вино на хорошее и плохое. Хорошим вином будем называть то, параметр `quality` которого — 6 и более.   
Сравните несколько методов классификации: ***логистическую регрессию***, ***дерево решений*** и ***бэггинг***. Это позволит вам увидеть, как меняется качество в зависимости от выбора того или иного алгоритма.   
    
Разделите выборку на обучающую и тестовую в соотношении `70/30`, в качестве значения параметра `random_state` возьмите число `42`.   
Для начала обучите два классификатора: 
- логистическую регрессию (с параметрами по умолчанию)   
- дерево решений (random_state = 42, максимальная глубина — 10).   

1. Введите значение F1-score для классификатора, который показал наилучшее значение. Ответ округлите до трёх знаков после точки-разделителя.

In [5]:
# получим и подготовим данные
df = pd.read_csv('./data/wineQualityReds.csv').drop(columns='Unnamed: 0')

In [6]:
df.quality = (df.quality>=6).astype(int)

X, y = df.drop(columns='quality'), df.quality

# scaler = StandardScaler()
# X = scaler.fit_transform(X)

In [7]:
y.sum()

855

In [8]:
# разделим выборку на обучающую и тестовую в соотношении 70/30, в качестве значения параметра 
# random_state возьмем число 42
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.3,random_state=42)

In [9]:
# логистическую регрессию
lr = LogisticRegression(solver='lbfgs',max_iter=1000)
lr.fit(X_train,y_train)
lr_pred = lr.predict(X_test)

# дерево решений  
dt = DecisionTreeClassifier(max_depth=10,random_state=42)
dt.fit(X_train,y_train)
dt_pred = dt.predict(X_test)

ans = max(f1_score(y_test,lr_pred), f1_score(y_test,dt_pred))
round(ans,3)

0.793

### Задание 2.8
Обучите модель с использование бэггинга (класс `BaggingClassifier` с `random_state=42`, разделение выборки на обучающую и тестовую — в соотношении 70/30 с random_state=42).

Возьмите из предыдущего задания алгоритм, показавший наилучшее качество, и укажите для него новое количество моделей — 1500. Вычислите новое значение F1-score.

In [10]:
bc = BaggingClassifier(
    base_estimator=dt,
    n_estimators=1500,
    random_state=42
)
bc.fit(X_train,y_train)
bc_pred = bc.predict(X_test)
ans = f1_score(y_test,bc_pred)
ans, round(ans,3)

(0.8150943396226414, 0.815)

## 3. Случайный лес

Давайте решим небольшую задачу .   
Мы будем анализировать набор данных **Boston Houses**, в котором объектами являются районы города, признаками — некие социальные и географические характеристики района, а целевой переменной — медианная стоимость домов в районе. Таким образом, мы будем решать задачу регрессии.   

Описание признаков:
- crim_rate — уровень преступности в районе;
- zn — доля участков площадью более 25 000 кв. футов;
- business — уровень развитости бизнеса в районе;
- river — наличие реки в районе;
- nit_oxiden — концентрация оксидов азота в воздухе;
- rooms — среднее число комнат в домах района;
- age — процент домов, построенных до 1940 года;
- dist — расстояние до центров занятости;
- highways_index — индекс доступности крупных дорог;
- tax — средняя ставка налога на имущество;
- pup_per_teac — среднее число учеников на одного учителя;
- lower — процент малообеспеченного населения в районе;
- target — медианная стоимость домов в районе (целевая переменная).

In [11]:
df = pd.read_csv('./data/boston.csv')

columns_to_convert = df.columns[df.dtypes == 'object']
df[columns_to_convert] = df[columns_to_convert].apply(lambda x: x.str.replace(',','.').astype(float))

In [12]:
X, y = df.drop(columns='target'), df.target

### Задание 3.4
1. Разбейте набор данных на обучающую и тестовую выборку в соотношении 70/30, при разбиении задайте параметр random_state = 13.

Какое получилось среднее значение медианных цен на обучающей выборке? Ответ округлите до двух знаков после точки-разделителя.

In [13]:
X_train, X_test,y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=13)

round(y_train.mean(),2)

22.77

2. Обучите линейную регрессию с параметрами по умолчанию.

В качестве ответа введите ошибку MAE на тестовой выборке. Ответ округлите до двух знаков после точки-разделителя.

In [14]:
lr = LinearRegression()
lr.fit(X_train,y_train)
lr_pred = lr.predict(X_test)
ans = mean_absolute_error(y_test,lr_pred)
round(ans,2)

3.72

3. Обучите решающее дерево с параметрами по умолчанию и аргументом random_state = 13.

Можно ли, опираясь на результаты, сделать вывод, что алгоритм переобучился?

In [15]:
dt = DecisionTreeRegressor(random_state = 13)
dt.fit(X_train,y_train)
dt_pred_train = dt.predict(X_train)
dt_pred_test = dt.predict(X_test)
mean_absolute_error(y_train,dt_pred_train), mean_absolute_error(y_test,dt_pred_test)

(0.0, 2.8026315789473686)

4. Обучите четыре случайных леса с числом деревьев 3, 10, 100, 500 и параметром random_state = 13.

В качестве ответа введите наименьшую полученную ошибку MAE на тестовой выборке. Ответ округлите до двух знаков после точки-разделителя.

In [16]:
n_estimators_row =[ 3, 10, 100, 500]
mae_row = np.array([])

for n_estimators in n_estimators_row:
    rfr = RandomForestRegressor(n_estimators=n_estimators,random_state = 13)
    rfr.fit(X_train,y_train)
    rfr_pred = rfr.predict(X_test)
    mae = mean_absolute_error(y_test,rfr_pred)
    mae_row = np.append(mae_row,mae)

round(mae_row.min(),2)

2.24

## 4. Случайный лес. Практика

Давайте посмотрим, с какими данными нам предстоит работать.  
    

In [17]:
df = pd.read_csv('./data/weatherAUS.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 145460 entries, 0 to 145459
Data columns (total 23 columns):
Date             145460 non-null object
Location         145460 non-null object
MinTemp          143975 non-null float64
MaxTemp          144199 non-null float64
Rainfall         142199 non-null float64
Evaporation      82670 non-null float64
Sunshine         75625 non-null float64
WindGustDir      135134 non-null object
WindGustSpeed    135197 non-null float64
WindDir9am       134894 non-null object
WindDir3pm       141232 non-null object
WindSpeed9am     143693 non-null float64
WindSpeed3pm     142398 non-null float64
Humidity9am      142806 non-null float64
Humidity3pm      140953 non-null float64
Pressure9am      130395 non-null float64
Pressure3pm      130432 non-null float64
Cloud9am         89572 non-null float64
Cloud3pm         86102 non-null float64
Temp9am          143693 non-null float64
Temp3pm          141851 non-null float64
RainToday        142199 non-null obje

Данные содержат 23 признака и 145 460 наблюдений. Из этих 23 признаков шесть — категориальные, в одном записана дата, а остальные являются непрерывными числовыми данными.

Примеры числовых признаков: температура, скорость ветра, влажность, облачность, атмосферное давление в разное время суток, количество осадков, испарение, количество часов с солнечной погодой.   
Примеры категориальных признаков: местоположение, направление ветра в разное время суток, наличие дождя сегодня или завтра.   
Целевой переменной является столбец RainTomorrow. Значение этой переменной мы и будем пытаться предсказать.

#### Подробная расшифровка всех признаков
- Date — дата, в которую зафиксировано наблюдение;
- Location — местонахождение метеорологической станции;
- MinTemp — минимальная температура (℃);
- MaxTemp — максимальная температура (℃);
- Rainfall — количество осадков (дождь) за сутки (мм);
- Evaporation — количество испарений до 9 утра (мм);
- Sunshine — количество часов в сутках, когда светило солнце;
- WindGustDir — направление самого сильного порыва ветра за последние 24 часа;
- WindGustSpeed — скорость самого сильного порыва ветра за последние 24 часа;
- WindDir9am — направление ветра в 9 утра;
- WindDir3pm — направление ветра в 3 часа дня;
- WindSpeed9am — скорость ветра в 9 часов утра;
- WindSpeed3pm — скорость ветра в 3 часа дня;
- Humidity9am — влажность в 9 утра;
- Humidity3pm — влажность в 3 часа дня;
- Pressure9am — атмосферное давление в 9 утра;
- Pressure3pm — атмосферное давление в 3 часа дня;
- Cloud9am — часть неба, закрытая облаками, в 9 утра;
- Cloud3pm — часть неба, закрытая облаками, в 3 часа дня;
- Temp9am — температура в 9 утра;
- Temp3pm — температура в 3 часа дня;
- RainToday — наличие дождя в этот день;
- RainTomorrow — наличие дождя на следующий день.

### Задание 4.1
Сколько суммарно пропусков в данных?

In [18]:
ans = df.isna().sum().sum()
print(f'Суммарно пропусков в данных {ans}')

Суммарно пропусков в данных 343248


### Задание 4.2
В некоторых признаках пропусков более 40 % — удалите такие признаки. Сколько их было?

In [19]:
del_mask = df.isna().sum()/df.shape[0]>0.4
to_del = df.columns[del_mask]
df.drop(columns=to_del,inplace=True)
print(f'Признаков с более 40% пропусков {len(to_del)}')

Признаков с более 40% пропусков 3


### Задание 4.3
Теперь обработаем признаки `RainToday` и `RainTomorrow` таким образом, чтобы вместо `yes` было значение `1`, а вместо `no` — значение `0`. Вычислите среднее арифметическое для преобразованного признака `RainToday` и запишите его в ответ, предварительно округлив до двух знаков после точки-разделителя.

In [20]:
df.RainToday = (df.RainToday == 'Yes').astype(int)
df.RainTomorrow = (df.RainTomorrow == 'Yes').astype(int)

ans = df.RainToday.mean()
print(f'Среднее арифметическое для преобразованного признака RainToday {ans:.2f}')

Среднее арифметическое для преобразованного признака RainToday 0.22


### Задание 4.4
Обработайте признак `Date` таким образом, чтобы выделить в отдельный признак `Month` (номер месяца). Изначальный признак `Date` удалите. Определите, в какой месяц выпадает больше всего дождей. В качестве ответа введите порядковый номер месяца.

In [21]:
df['Month'] = pd.to_datetime(df.Date).dt.month
df.drop(columns='Date', inplace=True)

ans = df.groupby('Month').RainToday.mean().idxmax()
print(f'В {ans} месяц выпадает больше всего дождей')

В 7 месяц выпадает больше всего дождей


### Задание 4.5
Обработайте оставшиеся категориальные признаки. С помощью метода `get_dummies` с настройками по умолчанию создайте dummy-переменные для всех категориальных признаков (их пять), которые есть в данных на этот момент.

Сколько теперь признаков в данных, если считать целевую переменную?

In [22]:
columns = ['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm','Month']
df_dummies = pd.get_dummies(df,columns=columns)

print(f'{df_dummies.shape[1]} теперь признаков в данных, если считать целевую переменную')

124 теперь признаков в данных, если считать целевую переменную


### Задание 4.6
Осталось совсем немного. Удалите все строки, где есть пропуски. Далее разбейте данные на обучающую и тестовую выборки в соотношении 70/30, в качестве значения параметра `random_state` возьмите число `31`.

Каково среднее значение целевой переменной на тестовой выборке? Ответ округлите до двух знаков после точки-разделителя.

In [23]:
df_dummies.dropna(axis='index',inplace=True)

X, y = df_dummies.drop(columns='RainTomorrow'), df_dummies.RainTomorrow
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,random_state=31 )

print(f'{y_test.mean():.2f} среднее значение целевой переменной на тестовой выборке')

0.23 среднее значение целевой переменной на тестовой выборке


### Задание 4.7
Теперь давайте вспомним про бутстреп. Он не понадобится нам для решения этой задачи, но будет полезно реализовать его «вручную».

Сделайте оценку стандартного отклонения для среднего значения минимальной температуры для обучающей выборки (то есть для среднего значения по признаку MinTemp). Для этого сгенерируйте 1000 случайных выборок из наших данных — каждая из них должна быть такого же объёма, как и обучающая выборка. Для генерации выборки используйте np.random.randint(): сгенерируйте необходимое количество индексов и по ним извлеките соответствующие элементы выборки. Случайность фиксируйте с помощью np.random.seed(31).

Для каждой выборки вычислите среднее значение, а после найдите стандартное отклонение для этих значений. Ответ округлите до двух знаков после точки-разделителя.

In [24]:
np.random.seed(31)

rows_mum = X_train.shape[0]
inds = np.random.randint(0,rows_mum,(1000,rows_mum))

means = [x.mean() for x in X_train.MinTemp.values[inds]]
ans = np.std(means)
round(ans,2)

0.03

### Задание 4.8
Теперь можно перейти к обучению прогностических моделей. Начнём с того, что построим простейшую `логистическую регрессию (без настройки гиперпараметров)`. Это будет та модель, с качеством которой мы будем сравнивать результаты, полученные далее, чтобы оценить превосходство случайного леса над простыми методами.

В качестве ответа введите значение метрики `roc_auc` на тестовой выборке. Ответ округлите до двух знаков после точки-разделителя.

In [25]:
lr = LogisticRegression(solver='lbfgs')
lr.fit(X_train, y_train)
lr_pred = lr.predict(X_test)

ans = roc_auc_score(y_test,lr_pred)
round(ans,2)

C:\Users\GANSOR-PC\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


0.73

### Задание 4.9
Теперь попробуйте обучить на наших данных другой алгоритм — дерево решений. С помощью `GridSearchCV` сделайте перебор гиперпараметров по следующей сетке:

```
params = {'max_leaf_nodes': list(range(2, 10)), 'min_samples_split': [2, 3, 4], 'max_depth': [5,7,9,11]}
```
Для параметра кросс-валидации `cv` задайте значение `3`. Для решающего дерева определите параметр `random_state=42`. Остальные параметры оставьте по умолчанию.

In [26]:
%%time
dt = DecisionTreeClassifier(random_state=42)
# param_grid = {'max_leaf_nodes': list(range(2, 10)), 
#           'min_samples_split': [2, 3, 4], 
#           'max_depth': [5,7,9,11]}
param_grid = {'max_leaf_nodes': list(range(5, 15)), 
              'min_samples_split': [2, 3, 4], 
              'max_depth': [5,7,9,11]}
search = GridSearchCV(estimator=dt,
                      param_grid=param_grid,
                      n_jobs=-1,
                      cv=3)
search.fit(X_train,y_train)
best_dt = search.best_estimator_

Wall time: 20.2 s


1. Вычислите значение `roc_auc` для решающего дерева с гиперпараметрами, определёнными в качестве оптимальных. Ответ округлите до двух знаков после точки-разделителя.

In [27]:
best_dt_pred = best_dt.predict(X_test)
ans = roc_auc_score(y_test,best_dt_pred)
round(ans,2)

0.71

2. Какие значения гиперпараметров были выбраны как оптимальные?


In [28]:
search.best_params_

{'max_depth': 5, 'max_leaf_nodes': 12, 'min_samples_split': 2}

### Задание 4.10
К сожалению, деревья решений не помогли нам в улучшении качества модели, так что попробуем ещё уменьшить ошибку с помощью ансамблей.

Теперь постройте `случайный лес, включающий 100 деревьев`. Задайте параметр `random_state=31`. Остальные параметры оставьте по умолчанию.

Какой теперь будет метрика roc_auc на тестовой выборке? Ответ округлите до двух знаков после точки-разделителя.

In [29]:
%%time
rf = RandomForestClassifier(n_estimators=100,
                            random_state=31)
rf.fit(X_train,y_train)
rf_pred = rf.predict(X_test)
roc_auc_score(y_test,rf_pred), roc_auc_score(rf_pred,y_test)

Wall time: 11.2 s


(0.7360027958675083, 0.8211798573558224)

### Задание 4.11
Основные параметры, которые отвечают за качество обучения в случайном лесе, следующие:`'max_features'`, `'min_samples_leaf'`, `'max_depth'`.

Возьмите случайный лес из 100 деревьев и найдите оптимальную комбинацию этих трёх параметров. Сетка для перебора следующая:
```python
{'max_features': [ 4, 5, 6, 7], 'min_samples_leaf': [3, 5, 7, 9, 11], 'max_depth': [5, 10, 15]}
```
Перебор осуществите с помощью `GridSearchCV`. Для параметра кросс-валидации `cv задайте значение 3`. Случайности фиксируйте параметром `random_state = 31`. Остальные значения оставьте по умолчанию.

Какое значение `roc_auc` получилось для оптимальных гиперпараметров?

In [30]:
%%time
rfc = RandomForestClassifier(n_estimators=100,random_state=31)
param_grid = {'max_features': [ 4, 5, 6, 7], 
              'min_samples_leaf': [3, 5, 7, 9, 11], 
              'max_depth': [5, 10, 15]}
search = GridSearchCV(estimator=rfc,
                     param_grid=param_grid,
                     n_jobs=-1,
                     cv=3)
search.fit(X_train,y_train)
best_rfc = search.best_estimator_
best_rfc_pred = best_rfc.predict(X_test)
roc_auc_score(y_test,best_rfc_pred), roc_auc_score(best_rfc_pred,y_test)

Wall time: 1min 31s


(0.6985647172231163, 0.8233759612850785)

### Задание 4.12
Как мы говорили в предыдущем юните, благодаря случайному лесу можно узнать, какие признаки оказывают большее влияние на целевую переменную по сравнению с другими.

Оцените значимость признаков. Отметьте три признака, которые дают наибольший вклад в целевую переменную:


In [31]:
feature_importances = pd.Series(data = best_rfc.feature_importances_, index=X.columns)
# feature_importances.sort_values(ascending=False)[:3].index
print(*feature_importances.sort_values(ascending=False)[:3].index)

Humidity3pm Rainfall Cloud9am


In [32]:
feature_importances.sort_values(ascending=False)

Humidity3pm               0.249056
Rainfall                  0.084987
Cloud9am                  0.068340
Pressure3pm               0.065046
Humidity9am               0.062901
                            ...   
Location_BadgerysCreek    0.000000
Location_SalmonGums       0.000000
Location_Dartmoor         0.000000
Location_GoldCoast        0.000000
Location_Albany           0.000000
Length: 123, dtype: float64

### 5. Бустинг
    
#### ADABOOST
Первый ансамбль типа бустинг, который мы рассмотрим, называется `AdaBoost (Adaptive Boosting)`.

#### ГРАДИЕНТНЫЙ БУСТИНГ
Переходим к следующему алгоритму, который очень популярен благодаря своей высокой эффективности, — к `градиентному бустингу`.

### 6. Бустинг. Практика
Мы будем работать c данными, которые содержат результаты опроса клиентов авиакомпании по поводу их удовлетворённости полётом.

Нашей задачей будет предсказать удовлетворённость пассажиров.

Признаки в данных:   
- Gender — пол пассажира (женский, мужской);
- Customer Type — тип клиента (постоянный/непостоянный клиент);
- Age — возраст клиента;
- Type of Travel — цель перелета (личная/деловая поездка);
- Class — туристический класс пассажира (Business, Eco, Eco Plus);
- Flight distance — расстояние полета;
- Inflight wifi service — уровень удовлетворённости Wi-Fi (0 — не применимо, 1–5);
- Departure/Arrival time convenient — уровень удовлетворённости временем отправления и прибытия;
- Ease of Online booking — уровень удовлетворённости онлайн-бронированием;
- Gate location — уровень удовлетворённости расположением выхода на посадку;
- Food and drink — уровень удовлетворённости едой и напитками;
- Online boarding — уровень удовлетворённости онлайн-регистрацией;
- Seat comfort — уровень удовлетворённости комфортом сидений;
- Inflight entertainment — уровень удовлетворённости развлечениями на борту;
- On-board service — уровень удовлетворённости сервисом на борту;
- Leg room service — уровень удовлетворённости местом для ног;
- Baggage handling — уровень удовлетворённости обработкой багажа;
- Check-in service — уровень удовлетворённости услугами регистрации;
- Inflight service — уровень удовлетворённости обслуживанием во время полёта;
- Cleanliness — уровень удовлетворённости чистотой;
- Departure Delay in Minutes — задержка при отправлении (в минутах);
- Arrival Delay in Minutes — задержка при прибытии (в минутах);
- Satisfaction — удовлетворённость авиакомпанией — целевая переменная (satisfaction/neutral/dissatisfaction).

In [33]:
df = pd.read_csv('./data/AirPass.csv',index_col=0)

In [34]:
df.describe(include='all').T.assign(nunique = df.nunique(),dtypes=df.dtypes)

,count,unique,top,freq,mean,std,min,25%,50%,75%,max,nunique,dtypes
id,103904,NaN,NaN,NaN,64924.2,37463.8,1,32533.8,64856.5,97368.2,129880,103904,int64
Gender,103904,2,Female,52727,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,object
Customer Type,103904,2,Loyal Customer,84923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,object
Age,103904,NaN,NaN,NaN,39.3797,15.115,7,27,40,51,85,75,int64
Type of Travel,103904,2,Business travel,71655,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,object
Class,103904,3,Business,49665,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3,object
Flight Distance,103904,NaN,NaN,NaN,1189.45,997.147,31,414,843,1743,4983,3802,int64
Inflight wifi service,103904,NaN,NaN,NaN,2.72968,1.32783,0,2,3,4,5,6,int64
Departure/Arrival time convenient,103904,NaN,NaN,NaN,3.0603,1.52508,0,2,3,4,5,6,int64
Ease of Online booking,103904,NaN,NaN,NaN,2.7569,1.39893,0,2,3,4,5,6,int64


### Задание 6.1
Для начала сделаем небольшую предобработку данных. Сколько всего в данных пропущенных значений?

In [35]:
ans = df.isna().sum().sum()
print(f'всего в данных {ans} пропущенных значений')

всего в данных 310 пропущенных значений


### Задание 6.2
Теперь давайте избавимся от найденных пропусков. Заполните их все медианными значениями. После этого вычислите среднее арифметическое для признака, отражающего задержку при прибытии в минутах. Ответ округлите до двух знаков после точки-разделителя.

In [36]:
df.fillna(value=df.median(),inplace=True)
ans = df['Arrival Delay in Minutes'].mean()
print(f'{ans:.2f}')

15.13


### Задание 6.3
Проведём небольшой разведывательный анализ. Посмотрим, в каких категориях пассажиров превалировали удовлетворённые полетом клиенты.

1. Сравните удовлетворённость полётом мужчин и женщин. Выберите верное утверждение:

In [37]:
df.groupby('Gender').satisfaction.value_counts()

Gender  satisfaction           
Female  neutral or dissatisfied    30193
        satisfied                  22534
Male    neutral or dissatisfied    28686
        satisfied                  22491
Name: satisfaction, dtype: int64

2. Теперь сравним категории пассажиров в зависимости от их цели поездки. Выберите все верные утверждения:

In [38]:
df.groupby('Type of Travel').satisfaction.value_counts()

Type of Travel   satisfaction           
Business travel  satisfied                  41746
                 neutral or dissatisfied    29909
Personal Travel  neutral or dissatisfied    28970
                 satisfied                   3279
Name: satisfaction, dtype: int64

3. В каком туристическом классе наибольший процент довольных клиентов?

In [39]:
df.groupby('Class').satisfaction.value_counts(normalize=True)

Class     satisfaction           
Business  satisfied                  0.694251
          neutral or dissatisfied    0.305749
Eco       neutral or dissatisfied    0.813862
          satisfied                  0.186138
Eco Plus  neutral or dissatisfied    0.753936
          satisfied                  0.246064
Name: satisfaction, dtype: float64

In [40]:
# Перекодируем часть бинарных признаков, чтобы использовать их при обучении:

df['satisfaction'] = df['satisfaction'].map({'neutral or dissatisfied':0 , 'satisfied':1})
df['Customer Type'] = df['Customer Type'].map({'Loyal Customer':1, 'disloyal Customer':0})
df['Type of Travel'] = df['Type of Travel'].map({'Personal Travel':0, 'Business travel':1})

### Задание 6.4
Для остальных категориальных признаков создайте dummy-переменные. Сделайте это с помощью функции `get_dummies()` из библиотеки `Pandas`, параметры не меняйте. Сколько теперь признаков в данных (включая целевую переменную)?

In [41]:
df=pd.get_dummies(df)
df.shape[1]

27

### Задание 6.5
Мы практически добрались до обучения модели. Разбейте данные на обучающую и тестовую выборки в соотношении 80/20, параметр `random_state = 26`. Сколько наблюдений попало в тестовую выборку?

In [42]:
X,y = df.drop(columns='satisfaction'),df.satisfaction
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=26)

print(f'{X_test.shape[0]} наблюдений попало в тестовую выборку')

20781 наблюдений попало в тестовую выборку


### Задание 6.6
Теперь нам необходимо реализовать масштабирование данных. Для этого обучите на обучающей выборке метод `StandardScaler()` и с помощью него преобразуйте и обучающую, и тестовую выборки. Не забудьте, что целевую переменную обрабатывать не нужно.

```
Примечание. Отметим, что если бы дальше мы работали только с деревьями, масштабирование бы не требовалось. Однако мы реализуем его, чтобы можно было обучать и другие модели и сравнивать полученные результаты.
```

В качестве ответа введите самое первое значение из матрицы преобразованных признаков тестовой выборки. Округлите значение до двух знаков после точки-разделителя.

In [43]:
scaller = StandardScaler()
X_train = scaller.fit_transform(X_train)
X_test = scaller.transform(X_test)

In [44]:
ans = X_test[0,0]
round(ans,2)

0.94

### Задание 6.7
Перейдём к обучению моделей. В качестве первой модели возьмём самую простую — `логистическую регрессию`. Мы делаем это для того, чтобы потом сравнивать с ней полученные результаты: так вы сможете выяснить, насколько ансамбли смогут улучшить точность прогноза.

Обучите логистическую регрессию с параметрами по умолчанию на наших данных. В качестве ответа введите значение метрики `f1_score`. Ответ округлите до трёх знаков после точки-разделителя.

In [45]:
lr = LogisticRegression()
lr.fit(X_train,y_train)
lr_pred = lr.predict(X_test)
ans = f1_score(y_test,lr_pred)
round(ans,3)

C:\Users\GANSOR-PC\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.855

### Задание 6.8
Теперь перейдём к бустингу. Начнём с обучения первой модели — `AdaBoost`. В качестве базовой модели для неё возьмите `решающее дерево` с параметром `random_state = 26`.

Обучите `AdaBoost`, зафиксировав `random_state` со значением `26` и задав `темп обучения 0.01`. В качестве ответа введите значение метрики `f1_score`. Ответ округлите до трёх знаков после точки-разделителя.

In [48]:
%%time
dt = DecisionTreeClassifier(random_state=26)
ab = AdaBoostClassifier(base_estimator=dt,learning_rate=0.01,random_state=26)
ab.fit(X_train,y_train)
ab_pred = ab.predict(X_test)
ans = f1_score(y_test,ab_pred)
round(ans,3)

Wall time: 578 ms


0.941

### Задание 6.9
Перейдем к следующему алгоритму — `градиентному бустингу`.

Будем настраивать количество деревьев и темп обучения, делая перебор по следующей сетке:
```python
params = {"n_estimators":2**np.arange(8), "learning_rate":0.1**np.arange(3)}
```
Используйте для поиска оптимальных параметров `GridSearchCV`, а для ускорения работы алгоритма задайте `параметр кросс-валидации, равный 3`.

Какое наибольшее значение метрики `f1_score` получилось? Ответ округлите до трёх знаков после точки-разделителя.

In [56]:
%%time
gbc = GradientBoostingClassifier()

param_grid = {"n_estimators":2**np.arange(8), 
              "learning_rate":0.1**np.arange(3)}
search = GridSearchCV(estimator=gbc,
                      param_grid=param_grid,
                      scoring='f1',
                      n_jobs=-1,
                      cv=3)

search.fit(X_train,y_train)

ans = search.best_score_
round(ans,3)

Wall time: 1min 38s


0.948

### Задание 6.10
Обучите алгоритм `XGBoost`. Так как он достаточно мощный «из коробки», определите его с параметрами по умолчанию. Задайте только `random_state = 26`. Какое значение метрики `f1_score` получилось? Ответ округлите до трёх знаков после точки-разделителя.

>Подсказка. Для того чтобы обучить алгоритм `XGBoost` для решения задачи классификации, вам понадобится `XGBClassifier` из библиотеки `xgboost`, установленной ранее. Вся дальнейшая последовательность действий (обучение модели, предсказание, оценка качества) идентична другим алгоритмам, например логистической регрессии.


In [69]:
xgb = XGBClassifier(random_state = 26, use_label_encoder=False)
# xgb = XGBClassifier(random_state = 26)

xgb.fit(X_train,y_train)

[16:46:07] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=6,
              num_parallel_tree=1, objective='binary:logistic',
              predictor='auto', random_state=26, reg_alpha=0, reg_lambda=1,
              scale_pos_weight=1, subsample=1, tree_method='exact',
              use_label_encoder=False, validate_parameters=1, verbosity=None)

In [70]:
xgb_pred = xgb.predict(X_test)
ans = f1_score(y_test,xgb_pred)
round(ans,3)

0.958

### Задание 6.11
Обучите алгоритм `CatBoost`. Как и `XGBoost`, будем обучать его с настройками по умолчанию. Какое значение метрики `f1_score` получилось? Ответ округлите до трёх знаков после точки-разделителя.

>Подсказка. Для того чтобы обучить алгоритм `CatBoost`, вам понадобится `CatBoostClassifier()` из библиотеки `catboost`, установленной ранее. Вся дальнейшая последовательность действий (обучение модели, предсказание, оценка качества) идентична другим алгоритмам, например логистической регрессии.   

In [76]:
cb = CatBoostClassifier(random_state = 26,verbose=False)
cb.fit(X_train,y_train)
cb_pred = cb.predict(X_test)
ans = f1_score(y_test,cb_pred)
round(ans,3)

0.96

### Задание 6.12
Выведите матрицу ошибок для алгоритма, который получил наилучшие показатели качества модели на обучающей выборке (будем считать, что оцениваем по `f1_score`). Заполните соответствующие значения:

In [79]:
print('Score  ab:',f1_score(y_train,ab.predict(X_train)))
print('Score  xgb:',f1_score(y_train,xgb.predict(X_train)))
print('Score  cb:',f1_score(y_train,cb.predict(X_train)))

Score  ab: 1.0
Score  xgb: 0.9752680003376382
Score  cb: 0.9741014278680452


In [82]:
print('Score  ab:',f1_score(y_test,ab_pred))
print('Score  xgb:',f1_score(y_test,xgb_pred))
print('Score  cb:',f1_score(y_test,cb_pred))

Score  ab: 0.9413314307666978
Score  xgb: 0.9579785161685312
Score  cb: 0.9604048492937383


In [91]:
print(confusion_matrix(y_test,cb_pred))

cb_train_pred = cb.predict(X_train)
print(confusion_matrix(y_train,cb_train_pred))

[[11434   239]
 [  473  8635]]
[[46660   546]
 [ 1295 34622]]


In [89]:
cb = CatBoostClassifier(random_state = 26,verbose=False)

train_dataset = Pool(data=X_train,label=y_train)
test_dataset = Pool(data=X_test,label=y_test)

cb.fit(train_dataset)

cm_train = get_confusion_matrix(cb, train_dataset)
print(cm_train)
cm_test = get_confusion_matrix(cb, test_dataset)
print(cm_test)

[[46660.   546.]
 [ 1295. 34622.]]
[[11434.   239.]
 [  473.  8635.]]


Задание 6.13
Оцените важность признаков для модели из предыдущего задания. Отметьте три признака, которые оказывают наибольшее влияние на значение целевой переменной:

In [109]:
feature_importances = pd.Series(index=X.columns,data=cb.feature_importances_)
ans = feature_importances.sort_values(ascending=False).index[:3]
print(*ans,sep =' | ')

Inflight wifi service | Type of Travel | Online boarding


## 7. Стекинг

Мы будем работать с данными, которые содержат информацию о звуках, издаваемых лягушками, и характеристики этих звуков.

Необходимо классифицировать семейства лягушек в зависимости от особенностей их кваканья и прочих звуковых эффектов.

Будем решать задачу классификации по выявлению лягушек, которые относятся к семейству 'Dendrobatidae' (признак 'Family').

В качестве признаков, которые мы будем использовать для предсказания, необходимо взять все, кроме:
- 'Family' — семейство лягушек;
- 'Genus' — род лягушек;
- 'Species' — вид лягушек;
- 'RecordID' — ID записи.   

Все остальные признаки относятся к акустическим особенностям кваканья — они-то нам и понадобятся, чтобы определить, к какому семейству относится лягушка.   

Разделите выборку на обучающую и тестовую в соотношении `80/20`, параметр `random_state = 31`.

In [117]:
df = pd.read_csv('./data/Frogs_MFCCs.csv')

In [118]:
columns = ['Family','Genus','Species','RecordID']
X, y = df.drop(columns=columns), df.Family
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=31)

### Задание 7.5
Для начала обучите на наших данных `случайный лес` с `десятью решающими деревьями`. Воспользуйтесь параметрами по умолчанию. В качестве значения `random_state` возьмите число `42`.

Оцените значение $F_1$-меры и введите его в качестве ответа, предварительно округлив до двух знаков после точки-разделителя.

In [ ]:
rfc = RandomForestClassifier(n_estimators=10,random_state=42)
rfc.fit(X_train,y_train)
rfc_pred = rfc.predict(X_test)

In [127]:
ans = f1_score(y_test,rfc_pred,average='micro')
round(ans,2)

0.98

### Задание 7.6
Теперь попробуем улучшить качество нашего предсказания за счёт использования стекинга.

В качестве базовых моделей выберите следующие:

- `случайный лес` с десятью деревьями, `random_state = 31`;
- `KNN`, `количество соседей = 11`;
- `наивный байесовский классификатор` с параметрами по умолчанию, в качестве метода возьмите `GaussianNB()`.
В качестве метамодели выберите `логистическую регрессию`.

Обучите модели и сделайте предсказание целевой метки для тестового набора данных.

Рассчитайте $F_1$-меру для тестового набора данных и введите её в качестве ответа, предварительно округлив до двух знаков после точки-разделителя.

In [ ]:
# локально не работает, см. Colab - StackingClassifier_example.ipynb

estimators = [('rf', RandomForestClassifier(n_estimators=10, random_state=31)),
              ('knn', KNeighborsClassifier(n_neighbors= 11)),
              ('nb', GaussianNB())]
sc = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
sc.fit(X_train, y_train)
pred = sc.predict(X_test)
f1_score(y_test, pred,average='micro')

# ===================================

In [111]:
df.head()

,MFCCs_ 1,MFCCs_ 2,MFCCs_ 3,MFCCs_ 4,MFCCs_ 5,MFCCs_ 6,MFCCs_ 7,MFCCs_ 8,MFCCs_ 9,MFCCs_10,...,MFCCs_17,MFCCs_18,MFCCs_19,MFCCs_20,MFCCs_21,MFCCs_22,Family,Genus,Species,RecordID
0,1.0,0.152936,-0.105586,0.200722,0.317201,0.260764,0.100945,-0.150063,-0.171128,0.124676,...,-0.108351,-0.077623,-0.009568,0.057684,0.118680,0.014038,Leptodactylidae,Adenomera,AdenomeraAndre,1
1,1.0,0.171534,-0.098975,0.268425,0.338672,0.268353,0.060835,-0.222475,-0.207693,0.170883,...,-0.090974,-0.056510,-0.035303,0.020140,0.082263,0.029056,Leptodactylidae,Adenomera,AdenomeraAndre,1
2,1.0,0.152317,-0.082973,0.287128,0.276014,0.189867,0.008714,-0.242234,-0.219153,0.232538,...,-0.050691,-0.023590,-0.066722,-0.025083,0.099108,0.077162,Leptodactylidae,Adenomera,AdenomeraAndre,1
3,1.0,0.224392,0.118985,0.329432,0.372088,0.361005,0.015501,-0.194347,-0.098181,0.270375,...,-0.136009,-0.177037,-0.130498,-0.054766,-0.018691,0.023954,Leptodactylidae,Adenomera,AdenomeraAndre,1
4,1.0,0.087817,-0.068345,0.306967,0.330923,0.249144,0.006884,-0.265423,-0.172700,0.266434,...,-0.048885,-0.053074,-0.088550,-0.031346,0.108610,0.079244,Leptodactylidae,Adenomera,AdenomeraAndre,1


In [112]:
df.columns

Index(['MFCCs_ 1', 'MFCCs_ 2', 'MFCCs_ 3', 'MFCCs_ 4', 'MFCCs_ 5', 'MFCCs_ 6',
       'MFCCs_ 7', 'MFCCs_ 8', 'MFCCs_ 9', 'MFCCs_10', 'MFCCs_11', 'MFCCs_12',
       'MFCCs_13', 'MFCCs_14', 'MFCCs_15', 'MFCCs_16', 'MFCCs_17', 'MFCCs_18',
       'MFCCs_19', 'MFCCs_20', 'MFCCs_21', 'MFCCs_22', 'Family', 'Genus',
       'Species', 'RecordID'],
      dtype='object')

# ===================================

In [113]:
columns

['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'Month']